# Dataset: FreeSolv


In [1]:
!pip install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 41.3 MB/s eta 0:00:00


In [2]:
%matplotlib inline
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import dgl.function as fn
import torch.nn.functional as F
import shutil
from torch.utils.data import DataLoader
import cloudpickle
from dgl.nn import GraphConv

#### Set Path

In [3]:
current_dir = "./"
checkpoint_path = current_dir + "save_models/model_checkpoints/" + "checkpoint"
os.makedirs(checkpoint_path, exist_ok=True)

best_model_path = current_dir + "save_models/best_model/"

folder_data_temp = current_dir +"data_temp/"
shutil.rmtree(folder_data_temp, ignore_errors=True)

path_save = current_dir + "graph_data.zip"
shutil.unpack_archive(path_save, folder_data_temp)

#### Custom PyTorch Datasets

In [4]:
from sklearn import preprocessing

""" Regression Dataset """
class DGLDatasetReg(torch.utils.data.Dataset):
    def __init__(self, address, transform=None, train=False, scaler=None, scaler_regression=None):
        # Constructor method for the dataset
        # address: the file path of the dataset
        # transform: any data transformation to be applied to the dataset
        # train: a boolean parameter indicating if the dataset is for training or not
        # scaler: a scaler object from the `preprocessing` module of scikit-learn for normalization
        # scaler_regression: a boolean parameter indicating if normalization should be applied to the labels or not
        self.train = train
        self.scaler = scaler
        # Load the dataset from the given file path
        self.data_set, train_labels_masks_globals = dgl.load_graphs(address + ".bin")
        num_graphs = len(self.data_set)
        # Extract the labels, masks and global features from the dataset
        self.labels = train_labels_masks_globals["labels"].view(num_graphs, -1)
        self.masks = train_labels_masks_globals["masks"].view(num_graphs, -1)
        self.globals = train_labels_masks_globals["globals"].view(num_graphs, -1)
        self.transform = transform
        self.scaler_regression = scaler_regression

    def scaler_method(self):
        # Method to fit a scaler on the training labels and return it
        # Returns None if the dataset is not for training
        if self.train:
            scaler = preprocessing.StandardScaler().fit(self.labels)
            self.scaler = scaler
        return self.scaler

    def __len__(self):
        # Method to return the length of the dataset
        return len(self.data_set)

    def __getitem__(self, idx):
        # Method to return a single sample from the dataset
        # idx: the index of the sample to be returned
        if self.scaler_regression:
            """ With Scaler"""
            # If scaler_regression is True, apply normalization to the labels using the fitted scaler
            return self.data_set[idx], torch.tensor(self.scaler.transform(self.labels)[idx]).float(), self.masks[idx], self.globals[idx]
        else:
            """ Without Scaler """
            # If scaler_regression is False, return the labels without normalization
            return self.data_set[idx], self.labels[idx].float(), self.masks[idx], self.globals[idx]

#### Defining Train, Validation, and Test Set

In [5]:
path_data_temp = folder_data_temp + "scaffold"+"_"+str(0)
train_set = DGLDatasetReg(address=path_data_temp+"_train")
val_set = DGLDatasetReg(address=path_data_temp+"_val")
test_set = DGLDatasetReg(address=path_data_temp+"_test")

print(len(train_set), len(val_set), len(test_set))


513 64 65


#### Data Loader

In [6]:
def collate(batch):
    # This function takes a list of tuples (graphs, labels, masks, globals) as input
    # Concatenate a sequence of graphs into a batched graph using `dgl.batch`
    graphs = [e[0] for e in batch]
    g = dgl.batch(graphs)

    # Concatenate a sequence of tensors (labels) along a new dimension
    # If a scaler object is provided, apply normalization to the labels using the scaler
    # Otherwise, return the labels without normalization
    if scaler is not None:
        labels = [torch.tensor(scaler.transform(e[1].unsqueeze(0)).squeeze(0)).float() for e in batch]
    else:
        labels = [e[1].float() for e in batch]
    labels = torch.stack(labels, 0)

    # Concatenate a sequence of tensors (masks) along a new dimension
    masks = [e[2] for e in batch]
    masks = torch.stack(masks, 0)

    # Concatenate a sequence of tensors (globals) along a new dimension
    globals_ = [e[3] for e in batch]
    globals_ = torch.stack(globals_, 0)

    # Return the batched graph, labels, masks, and globals
    return g, labels, masks, globals_

def loader(batch_size):
    # This function returns a set of data loaders for the training, validation, and test sets
    train_dataloader = DataLoader(train_set,
                              batch_size=batch_size,
                              collate_fn=collate,
                              drop_last=False,
                              shuffle=True,
                              num_workers=1)

    val_dataloader =  DataLoader(val_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)

    test_dataloader = DataLoader(test_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)
    return train_dataloader, val_dataloader, test_dataloader

In [7]:
train_dataloader, val_dataloader, test_dataloader = loader(batch_size=32)

#### Defining A GNN

##### Some Variables

In [8]:
#FreeSolv dataset has 1 task. Some other datasets may have some more number of tasks, e.g., tox21 has 12 tasks.
num_tasks = 1

# Size of global feature of each graph
global_size = 200

# Number of epochs to train the model
num_epochs = 100

# Number of steps to wait if the model performance on the validation set does not improve
patience = 10

#Configurations to instantiate the model
config = {"node_feature_size":127, "edge_feature_size":12, "hidden_size":100}


#GNN 1

In [ ]:
class GNN1(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size,allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.num_tasks,allow_zero_in_degree=True)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

#Function to Compute Score of the Model

In [9]:
import math
from sklearn.preprocessing import StandardScaler

def compute_score(model, data_loader, scaler, val_size, num_tasks=1):
    model.eval()
    loss_mean = nn.MSELoss(reduction='sum') # MSE with sum instead of mean, i.e., sum_i[(y_i)^2-(y'_i)^2]
    final_loss = 0
    if not isinstance(scaler, StandardScaler):
        scaler = StandardScaler()
        scaler.fit(data_loader.dataset.labels.numpy())

    with torch.no_grad():
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            if scaler is not None:
                prediction = torch.tensor(scaler.inverse_transform(prediction.detach().cpu()))
                labels = torch.tensor(scaler.inverse_transform(labels.cpu()))
            loss = loss_mean(prediction, labels)
            final_loss += loss.item()
        final_loss /= val_size
        final_loss = math.sqrt(final_loss) # RMSE
    return final_loss / num_tasks

#Loss Function

In [10]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    criterion = nn.MSELoss(reduction='none')
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

#### Training and Evaluation

##### Training Function

In [11]:
def train_epoch(train_dataloader, model, optimizer, num_tasks):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        optimizer.zero_grad(set_to_none=True)
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [12]:
def train_evaluate():

    model = GNN1(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer, num_tasks)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(best_val), "\n")


##### Function to compute test set score of the final saved model

In [13]:
def test_evaluate():
    final_model = GNN1(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

##### Train the model and evaluate its performance

In [ ]:
import time
start_time = time.time()

scaler = StandardScaler()
train_labels = train_set.labels.numpy()
scaler.fit(train_labels)

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.852 | Valid Score: 4.304
 
Epoch: 1/100 | Best Valid Score Until Now: 4.304 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.762 | Valid Score: 3.998
 
Epoch: 2/100 | Best Valid Score Until Now: 3.998 

Patience 1
Epoch: 3/100 | Training Loss: 0.696 | Valid Score: 4.112
 
Epoch: 3/100 | Best Valid Score Until Now: 3.998 

Patience 2
Epoch: 4/100 | Training Loss: 0.620 | Valid Score: 4.084
 
Epoch: 4/100 | Best Valid Score Until Now: 3.998 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.875 | Valid Score: 3.970
 
Epoch: 5/100 | Best Valid Score Until Now: 3.970 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.592 | Valid Score: 3.950
 
Epoch: 6/100 | Best Valid Score Until Now: 3.950 

Patience 1
Epoch: 7/100 | Training Loss: 0.522 | Valid Score: 4.056
 
Epoch: 7/100 | Best Valid Score Until Now: 3.950 

Patience 2
Epoch: 8/100 | Training Loss: 0.494 | Valid Score: 4.028
 
Epoch: 8/100 | Best Valid Score Until Now: 3.950 

Patience 3
E

#GNN 2

In [ ]:
class GNN2(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size,allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size,allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.num_tasks,allow_zero_in_degree=True)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

In [ ]:
def train_evaluate():

    model = GNN2(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer, num_tasks)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(best_val), "\n")


##### Function to compute test set score of the final saved model

In [ ]:
def test_evaluate():
    final_model = GNN2(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

##### Train the model and evaluate its performance

In [ ]:
import time
start_time = time.time()

scaler = StandardScaler()
train_labels = train_set.labels.numpy()
scaler.fit(train_labels)

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.839 | Valid Score: 4.125
 
Epoch: 1/100 | Best Valid Score Until Now: 4.125 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.708 | Valid Score: 3.942
 
Epoch: 2/100 | Best Valid Score Until Now: 3.942 

Patience 1
Epoch: 3/100 | Training Loss: 0.645 | Valid Score: 3.963
 
Epoch: 3/100 | Best Valid Score Until Now: 3.942 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.565 | Valid Score: 3.875
 
Epoch: 4/100 | Best Valid Score Until Now: 3.875 

Patience 1
Epoch: 5/100 | Training Loss: 0.534 | Valid Score: 3.880
 
Epoch: 5/100 | Best Valid Score Until Now: 3.875 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.569 | Valid Score: 3.820
 
Epoch: 6/100 | Best Valid Score Until Now: 3.820 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.481 | Valid Score: 3.761
 
Epoch: 7/100 | Best Valid Score Until Now: 3.761 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.486 | Valid Score: 3.747
 
Epoch: 8/100 | Best Valid Score Until Now: 3.747 

Pa

#GNN 3

In [ ]:
class GNN3(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)


        self.conv1 = GraphConv(self.node_feature_size, 64 ,allow_zero_in_degree=True)
        self.conv2 = GraphConv(64, 128,allow_zero_in_degree=True)
        self.conv3 = GraphConv(128, 256,allow_zero_in_degree=True)
        self.conv4 = GraphConv(256, 128,allow_zero_in_degree=True)
        self.conv5 = GraphConv(128, self.num_tasks,allow_zero_in_degree=True)

        self.dropout = nn.Dropout(p=0.0)
        #self.bn1 = nn.BatchNorm1d(64)
        #self.bn2 = nn.BatchNorm1d(128)
        #self.bn3 = nn.BatchNorm1d(256)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        #h = self.bn1(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        #h = self.bn2(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        #h = self.bn3(h)
        h = self.conv4(mol_dgl_graph, h)
        h = F.relu(h)
        #h = self.bn2(h)
        h = self.conv5(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

In [ ]:
def train_evaluate():

    model = GNN3(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer, num_tasks)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(best_val), "\n")


##### Function to compute test set score of the final saved model

In [ ]:
def test_evaluate():
    final_model = GNN3(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

##### Train the model and evaluate its performance

In [ ]:
import time
start_time = time.time()

scaler = StandardScaler()
train_labels = train_set.labels.numpy()
scaler.fit(train_labels)

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.895 | Valid Score: 4.237
 
Epoch: 1/100 | Best Valid Score Until Now: 4.237 

Patience 1
Epoch: 2/100 | Training Loss: 0.684 | Valid Score: 4.292
 
Epoch: 2/100 | Best Valid Score Until Now: 4.237 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.642 | Valid Score: 3.829
 
Epoch: 3/100 | Best Valid Score Until Now: 3.829 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.573 | Valid Score: 3.690
 
Epoch: 4/100 | Best Valid Score Until Now: 3.690 

Patience 1
Epoch: 5/100 | Training Loss: 0.488 | Valid Score: 4.077
 
Epoch: 5/100 | Best Valid Score Until Now: 3.690 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.542 | Valid Score: 3.680
 
Epoch: 6/100 | Best Valid Score Until Now: 3.680 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.467 | Valid Score: 3.674
 
Epoch: 7/100 | Best Valid Score Until Now: 3.674 

Patience 1
Epoch: 8/100 | Training Loss: 0.452 | Valid Score: 3.750
 
Epoch: 8/100 | Best Valid Score Until Now: 3.674 

Save ch

#Function to Compute Score of the Model

#Graph SAGE 1

In [ ]:
import dgl.function as fn
from dgl.nn import SAGEConv

class GraphSAGE(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size, aggregator_type='mean')
        self.conv2 = SAGEConv(self.hidden_size, self.num_tasks, aggregator_type='mean')

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

In [ ]:
def train_evaluate():

    model = GraphSAGE(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer, num_tasks)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(best_val), "\n")


##### Function to compute test set score of the final saved model

In [ ]:
def test_evaluate():
    final_model = GraphSAGE(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

##### Train the model and evaluate its performance

In [ ]:
import time
start_time = time.time()

scaler = StandardScaler()
train_labels = train_set.labels.numpy()
scaler.fit(train_labels)

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.838 | Valid Score: 4.692
 
Epoch: 1/100 | Best Valid Score Until Now: 4.692 

Patience 1
Epoch: 2/100 | Training Loss: 0.524 | Valid Score: 4.799
 
Epoch: 2/100 | Best Valid Score Until Now: 4.692 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.451 | Valid Score: 4.646
 
Epoch: 3/100 | Best Valid Score Until Now: 4.646 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.401 | Valid Score: 4.537
 
Epoch: 4/100 | Best Valid Score Until Now: 4.537 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.372 | Valid Score: 4.470
 
Epoch: 5/100 | Best Valid Score Until Now: 4.470 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.293 | Valid Score: 4.371
 
Epoch: 6/100 | Best Valid Score Until Now: 4.371 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.264 | Valid Score: 4.303
 
Epoch: 7/100 | Best Valid Score Until Now: 4.303 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.238 | Valid Score: 4.188
 
Epoch: 8/100 | Best Valid Score Until Now: 4.188

#Graph SAGE 2


In [ ]:
import dgl.function as fn
from dgl.nn import SAGEConv

class GraphSAGE2(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size, aggregator_type='mean')
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size , aggregator_type='mean')
        self.conv3 = SAGEConv(self.hidden_size, self.num_tasks, aggregator_type='mean')

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

In [ ]:
def train_evaluate():

    model = GraphSAGE2(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer, num_tasks)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(best_val), "\n")


##### Function to compute test set score of the final saved model

In [ ]:
def test_evaluate():
    final_model = GraphSAGE2(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

##### Train the model and evaluate its performance

In [ ]:
import time
start_time = time.time()

scaler = StandardScaler()
train_labels = train_set.labels.numpy()
scaler.fit(train_labels)

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 3.745 | Valid Score: 4.914
 
Epoch: 1/100 | Best Valid Score Until Now: 4.914 

Save checkpoint
Epoch: 2/100 | Training Loss: 1.031 | Valid Score: 3.718
 
Epoch: 2/100 | Best Valid Score Until Now: 3.718 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.563 | Valid Score: 3.668
 
Epoch: 3/100 | Best Valid Score Until Now: 3.668 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.393 | Valid Score: 3.505
 
Epoch: 4/100 | Best Valid Score Until Now: 3.505 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.325 | Valid Score: 3.435
 
Epoch: 5/100 | Best Valid Score Until Now: 3.435 

Patience 1
Epoch: 6/100 | Training Loss: 0.296 | Valid Score: 3.461
 
Epoch: 6/100 | Best Valid Score Until Now: 3.435 

Patience 2
Epoch: 7/100 | Training Loss: 0.279 | Valid Score: 3.553
 
Epoch: 7/100 | Best Valid Score Until Now: 3.435 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.223 | Valid Score: 3.385
 
Epoch: 8/100 | Best Valid Score Until Now: 3.385 

Pa

#Graph SAGE 3


In [ ]:
from dgl.nn.pytorch.conv import SAGEConv
class GraphSAGE3(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, 64 ,aggregator_type='lstm')
        self.conv2 = SAGEConv(64, 128,aggregator_type='lstm')
        self.conv3 = SAGEConv(128, 256,aggregator_type='lstm')
        self.conv4 = SAGEConv(256, 128,aggregator_type='lstm')
        self.conv5 = SAGEConv(128, self.num_tasks,aggregator_type='lstm')

        self.dropout = nn.Dropout(p=0.2)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(256)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        #h = self.bn1(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        #h = self.bn2(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.dropout(h)
        #h = self.bn3(h)
        h = self.conv4(mol_dgl_graph, h)
        h = F.relu(h)
        #h = self.bn2(h)
        h = self.conv5(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

In [ ]:
def train_evaluate():

    model = GraphSAGE3(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer, num_tasks)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(best_val), "\n")


##### Function to compute test set score of the final saved model

In [ ]:
def test_evaluate():
    final_model = GraphSAGE3(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

##### Train the model and evaluate its performance

In [ ]:
import time
start_time = time.time()

scaler = StandardScaler()
train_labels = train_set.labels.numpy()
scaler.fit(train_labels)

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.959 | Valid Score: 3.725
 
Epoch: 1/100 | Best Valid Score Until Now: 3.725 

Patience 1
Epoch: 2/100 | Training Loss: 0.523 | Valid Score: 3.957
 
Epoch: 2/100 | Best Valid Score Until Now: 3.725 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.390 | Valid Score: 3.669
 
Epoch: 3/100 | Best Valid Score Until Now: 3.669 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.304 | Valid Score: 2.793
 
Epoch: 4/100 | Best Valid Score Until Now: 2.793 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.204 | Valid Score: 2.645
 
Epoch: 5/100 | Best Valid Score Until Now: 2.645 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.193 | Valid Score: 2.304
 
Epoch: 6/100 | Best Valid Score Until Now: 2.304 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.201 | Valid Score: 1.896
 
Epoch: 7/100 | Best Valid Score Until Now: 1.896 

Patience 1
Epoch: 8/100 | Training Loss: 0.166 | Valid Score: 2.282
 
Epoch: 8/100 | Best Valid Score Until Now: 1.896 

Pa

# GAT 1

In [ ]:
from dgl.nn.pytorch import GATConv
class GATConv1(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        # Number of attention heads
        self.num_heads = self.config.get('num_heads', 1)

        # Dropout probability
        self.dropout = self.config.get('dropout', 0.0)

        # GAT layer
        self.conv1 = GATConv(
            self.node_feature_size,
            self.hidden_size,
            num_heads=self.num_heads,
            feat_drop=self.dropout,
            attn_drop=self.dropout,allow_zero_in_degree=True
        )

        # Linear layer
        self.fc = nn.Linear(
            self.hidden_size * self.num_heads,
            self.hidden_size
        )

        # GAT layer
        self.conv2 = GATConv(
            self.hidden_size,
            self.num_tasks,
            num_heads=1,
            feat_drop=self.dropout,
            attn_drop=self.dropout,allow_zero_in_degree=True
        )

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]

        # First GAT layer
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"]).flatten(1)
        h = F.relu(h)
        h = self.fc(h)
        h = F.dropout(h, p=self.dropout, training=self.training)

        # Second GAT layer
        h = self.conv2(mol_dgl_graph, h).squeeze(1)
        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")

In [ ]:
def train_evaluate():

    model = GATConv1(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer, num_tasks)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(best_val), "\n")


##### Function to compute test set score of the final saved model

In [ ]:
def test_evaluate():
    final_model = GATConv1(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

##### Train the model and evaluate its performance

In [ ]:
import time
start_time = time.time()

scaler = StandardScaler()
train_labels = train_set.labels.numpy()
scaler.fit(train_labels)

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.831 | Valid Score: 3.946
 
Epoch: 1/100 | Best Valid Score Until Now: 3.946 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.662 | Valid Score: 3.544
 
Epoch: 2/100 | Best Valid Score Until Now: 3.544 

Patience 1
Epoch: 3/100 | Training Loss: 0.534 | Valid Score: 3.805
 
Epoch: 3/100 | Best Valid Score Until Now: 3.544 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.481 | Valid Score: 3.350
 
Epoch: 4/100 | Best Valid Score Until Now: 3.350 

Patience 1
Epoch: 5/100 | Training Loss: 0.488 | Valid Score: 3.578
 
Epoch: 5/100 | Best Valid Score Until Now: 3.350 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.411 | Valid Score: 3.316
 
Epoch: 6/100 | Best Valid Score Until Now: 3.316 

Patience 1
Epoch: 7/100 | Training Loss: 0.374 | Valid Score: 3.383
 
Epoch: 7/100 | Best Valid Score Until Now: 3.316 

Patience 2
Epoch: 8/100 | Training Loss: 0.361 | Valid Score: 3.557
 
Epoch: 8/100 | Best Valid Score Until Now: 3.316 

Patience 3
E

# GAT 2

In [ ]:
from dgl.nn.pytorch import GATConv
class GATConv2(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        # Number of attention heads
        self.num_heads = self.config.get('num_heads', 1)

        # Dropout probability
        self.dropout = self.config.get('dropout', 0.0)

        # GAT layer
        self.conv1 = GATConv(self.node_feature_size,self.hidden_size,num_heads=self.num_heads,feat_drop=self.dropout,attn_drop=self.dropout,allow_zero_in_degree=True)

        # Linear layer
        self.fc = nn.Linear(self.hidden_size * self.num_heads,self.hidden_size)

        # GAT layer
        self.conv2 = GATConv(self.hidden_size,self.hidden_size,num_heads=1,feat_drop=self.dropout,attn_drop=self.dropout,allow_zero_in_degree=True)

        # GAT layer
        self.conv3 = GATConv(self.hidden_size,self.num_tasks,num_heads=1,feat_drop=self.dropout,attn_drop=self.dropout,allow_zero_in_degree=True)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]

        # First GAT layer
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"]).flatten(1)
        h = F.relu(h)
        h = self.fc(h)
        h = F.dropout(h, p=self.dropout, training=self.training)

        # Second GAT layer
        h = self.conv2(mol_dgl_graph, h).squeeze(1)
        h = F.relu(h)
        h = F.dropout(h, p=self.dropout, training=self.training)

        # Third GAT layer
        if self.num_tasks == 1:
            h = self.conv3(mol_dgl_graph, h).squeeze(1)
        else:
            hs = []
            for i in range(self.num_tasks):
                hi = self.conv3(mol_dgl_graph, h).squeeze(1)
                hs.append(hi)
            h = torch.stack(hs, dim=1)

        mol_dgl_graph.ndata["h"] = h

        if self.num_tasks == 1:
            return dgl.mean_nodes(mol_dgl_graph, "h")
        else:
            return dgl.mean_nodes(mol_dgl_graph, "h"), h

In [ ]:
def train_evaluate():

    model = GATConv2(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer, num_tasks)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(best_val), "\n")


##### Function to compute test set score of the final saved model

In [ ]:
def test_evaluate():
    final_model = GATConv2(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

##### Train the model and evaluate its performance

In [ ]:
import time
start_time = time.time()

scaler = StandardScaler()
train_labels = train_set.labels.numpy()
scaler.fit(train_labels)

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.844 | Valid Score: 4.449
 
Epoch: 1/100 | Best Valid Score Until Now: 4.449 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.670 | Valid Score: 4.281
 
Epoch: 2/100 | Best Valid Score Until Now: 4.281 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.635 | Valid Score: 4.155
 
Epoch: 3/100 | Best Valid Score Until Now: 4.155 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.585 | Valid Score: 4.100
 
Epoch: 4/100 | Best Valid Score Until Now: 4.100 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.594 | Valid Score: 3.968
 
Epoch: 5/100 | Best Valid Score Until Now: 3.968 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.581 | Valid Score: 3.941
 
Epoch: 6/100 | Best Valid Score Until Now: 3.941 

Patience 1
Epoch: 7/100 | Training Loss: 0.585 | Valid Score: 4.045
 
Epoch: 7/100 | Best Valid Score Until Now: 3.941 

Patience 2
Epoch: 8/100 | Training Loss: 0.516 | Valid Score: 4.096
 
Epoch: 8/100 | Best Valid Score Until Now: 3.941 

Pa

#GAT 3


In [ ]:
from dgl.nn.pytorch import GATConv
from torch.nn import BatchNorm1d

class GATConv3(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        # Number of attention heads
        self.num_heads = self.config.get('num_heads', 1)

        # Dropout probability
        self.dropout = self.config.get('dropout', 0.0)

        # GAT layer
        self.conv1 = GATConv(self.node_feature_size, 64, num_heads=self.num_heads,
                             feat_drop=self.dropout, attn_drop=self.dropout,
                             allow_zero_in_degree=True)
        self.conv2 = GATConv(64, 128, num_heads=1, feat_drop=self.dropout,
                             attn_drop=self.dropout, allow_zero_in_degree=True)
        self.conv3 = GATConv(128, 256, num_heads=1, feat_drop=self.dropout,
                             attn_drop=self.dropout, allow_zero_in_degree=True)
        self.conv4 = GATConv(256, 128, num_heads=1, feat_drop=self.dropout,
                             attn_drop=self.dropout, allow_zero_in_degree=True)
        self.conv5 = GATConv(128, self.num_tasks, num_heads=1, feat_drop=self.dropout,
                             attn_drop=self.dropout, allow_zero_in_degree=True)

        # Linear layer
        self.fc1 = nn.Linear(64 * self.num_heads, 64)
        self.bn1 = BatchNorm1d(64)
        self.fc2 = nn.Linear(128 * self.num_heads, 128)
        self.bn2 = BatchNorm1d(128)
        self.fc3 = nn.Linear(256 * self.num_heads, 256)
        self.bn3 = BatchNorm1d(256)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]

        # First GAT layer
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"]).flatten(1)
        h = F.relu(h)
        h = self.fc1(h)
        #h = self.bn1(h)  # add batch normalization
        h = F.dropout(h, p=self.dropout, training=self.training)

        # Second GAT layer
        h = self.conv2(mol_dgl_graph, h).squeeze(1)
        h = F.relu(h)
        h = self.fc2(h)
        #h = self.bn2(h)  # add batch normalization
        h = F.dropout(h, p=self.dropout, training=self.training)

        h = self.conv3(mol_dgl_graph, h).squeeze(1)
        h = F.relu(h)
        h = self.fc3(h)
        #h = self.bn3(h)  # add batch normalization
        h = F.dropout(h, p=self.dropout, training=self.training)

        h = self.conv4(mol_dgl_graph, h).squeeze(1)
        h = F.relu(h)
        h = self.fc2(h)
        #h = self.bn2(h)  # add batch normalization
        h = F.dropout(h, p=self.dropout, training=self.training)

        # Third GAT layer
        if self.num_tasks == 1:
            h = self.conv5(mol_dgl_graph, h).squeeze(1)
        else:
            hs = []
            for i in range(self.num_tasks):
                hi = self.conv3(mol_dgl_graph, h).squeeze(1)
                hs.append(hi)
            h = torch.stack(hs, dim=1)

        mol_dgl_graph.ndata["h"]= h

        if self.num_tasks == 1:
            return dgl.mean_nodes(mol_dgl_graph, "h")
        else:
            return dgl.mean_nodes(mol_dgl_graph, "h"), h

In [ ]:
def train_evaluate():

    model = GATConv3(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer, num_tasks)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(best_val), "\n")


##### Function to compute test set score of the final saved model

In [ ]:
def test_evaluate():
    final_model = GATConv3(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

##### Train the model and evaluate its performance

In [ ]:
import time
start_time = time.time()

scaler = StandardScaler()
train_labels = train_set.labels.numpy()
scaler.fit(train_labels)

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.925 | Valid Score: 4.272
 
Epoch: 1/100 | Best Valid Score Until Now: 4.272 

Patience 1
Epoch: 2/100 | Training Loss: 0.770 | Valid Score: 4.447
 
Epoch: 2/100 | Best Valid Score Until Now: 4.272 

Patience 2
Epoch: 3/100 | Training Loss: 0.660 | Valid Score: 4.399
 
Epoch: 3/100 | Best Valid Score Until Now: 4.272 

Patience 3
Epoch: 4/100 | Training Loss: 0.647 | Valid Score: 4.372
 
Epoch: 4/100 | Best Valid Score Until Now: 4.272 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.571 | Valid Score: 4.161
 
Epoch: 5/100 | Best Valid Score Until Now: 4.161 

Patience 1
Epoch: 6/100 | Training Loss: 0.630 | Valid Score: 4.378
 
Epoch: 6/100 | Best Valid Score Until Now: 4.161 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.531 | Valid Score: 4.148
 
Epoch: 7/100 | Best Valid Score Until Now: 4.148 

Patience 1
Epoch: 8/100 | Training Loss: 0.550 | Valid Score: 4.274
 
Epoch: 8/100 | Best Valid Score Until Now: 4.148 

Patience 2
Epoch:

#My architecture

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GraphClassifier(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks
        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        self.conv1 = nn.Conv2d(1, 128, kernel_size=(1, self.node_feature_size))
        self.bn1 = nn.BatchNorm2d(128) # Batch Normalization
        self.dropout1 = nn.Dropout(p=0.1) # Dropout
        self.conv2 = nn.Conv2d(128, 256, kernel_size=(1, 1))
        self.bn2 = nn.BatchNorm2d(256) # Batch Normalization
        self.dropout2 = nn.Dropout(p=0.1) # Dropout
        self.conv3 = nn.Conv2d(256,512,kernel_size=(1,1))
        self.bn3 = nn.BatchNorm2d(512) # Batch Normalization
        self.dropout3 = nn.Dropout(p=0.2) # Dropout
        self.conv4 = nn.Conv2d(512,256,kernel_size=(1,1))
        self.bn4 = nn.BatchNorm2d(256) # Batch Normalization
        self.dropout4 = nn.Dropout(p=0.1) # Dropout
        self.conv5 = nn.Conv2d(256,128,kernel_size=(1,1))
        self.bn5 = nn.BatchNorm2d(128) # Batch Normalization
        self.dropout5 = nn.Dropout(p=0.1) # Dropout
        self.fc1 = nn.Linear(128, 64)
        self.bn6 = nn.BatchNorm1d(64) # Batch Normalization
        self.dropout6 = nn.Dropout(p=0.0) # Dropout
        self.fc2 = nn.Linear(64, 32)
        self.bn7 = nn.BatchNorm1d(32) # Batch Normalization
        self.dropout7 = nn.Dropout(p=0.0) # Dropout
        self.fc3 = nn.Linear(32,num_tasks)

    def forward(self, mol_dgl_graph, globals):
        # The forward method takes a molecular graph and a global feature vector as input
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        # Truncate the node feature size to the specified size
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]
        # Truncate the edge feature size to the specified size
        x = mol_dgl_graph.ndata["v"].unsqueeze(1)
        x = x.view(-1, 1, mol_dgl_graph.number_of_nodes(), self.node_feature_size)
        x = self.conv1(x)
        #x = self.bn1(x) # Batch Normalization
        x = F.relu(x)
        x = self.dropout1(x) # Dropout
        x = self.conv2(x)
        #x = self.bn2(x) # Batch Normalization
        x = F.relu(x)
        x = self.dropout2(x) # Dropout
        x = self.conv3(x)
        #x = self.bn3(x) # Batch Normalization
        x = F.relu(x)
        x = self.dropout3(x) # Dropout
        x = self.conv4(x)
        #x = self.bn4(x) # Batch Normalization
        x = F.relu(x)
        x = self.dropout4(x) # Dropout
        x = self.conv5(x)
        #x = self.bn5(x) # Batch Normalization
        x = F.relu(x)
        x = self.dropout5(x) # Dropout
        x = torch.max(x, dim=-1)[0]
        x = x.view(-1, 128)
        x = self.fc1(x)
        #x = self.bn6(x) # Batch Normalization
        x = F.relu(x)
        x = self.dropout6(x) # Dropout
        x = self.fc2(x)
        #x = self.bn7(x) # Batch Normalization
        x = F.relu(x)
        x = self.dropout7(x) # Dropout
        x = self.fc3(x)
        mol_dgl_graph.ndata["h"] = x

        return dgl.mean_nodes(mol_dgl_graph, "h")

In [24]:
def train_evaluate():

    model = GraphClassifier(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    best_val = float('inf')
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer, num_tasks)
            model.eval()
            score_val = compute_score(model, val_dataloader, scaler, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(best_val), "\n")


##### Function to compute test set score of the final saved model

In [25]:
def test_evaluate():
    final_model = GraphClassifier(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, scaler, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

##### Train the model and evaluate its performance

In [32]:
start_time = time.time()

scaler = StandardScaler()
train_labels = train_set.labels.numpy()
scaler.fit(train_labels)

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.951 | Valid Score: 4.482
 
Epoch: 1/100 | Best Valid Score Until Now: 4.482 

Patience 1
Epoch: 2/100 | Training Loss: 0.974 | Valid Score: 4.504
 
Epoch: 2/100 | Best Valid Score Until Now: 4.482 

Patience 2
Epoch: 3/100 | Training Loss: 1.056 | Valid Score: 4.498
 
Epoch: 3/100 | Best Valid Score Until Now: 4.482 

Patience 3
Epoch: 4/100 | Training Loss: 0.963 | Valid Score: 4.531
 
Epoch: 4/100 | Best Valid Score Until Now: 4.482 

Patience 4
Epoch: 5/100 | Training Loss: 0.952 | Valid Score: 4.549
 
Epoch: 5/100 | Best Valid Score Until Now: 4.482 

Patience 5
Epoch: 6/100 | Training Loss: 0.978 | Valid Score: 4.505
 
Epoch: 6/100 | Best Valid Score Until Now: 4.482 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.966 | Valid Score: 4.473
 
Epoch: 7/100 | Best Valid Score Until Now: 4.473 

Patience 1
Epoch: 8/100 | Training Loss: 0.944 | Valid Score: 4.496
 
Epoch: 8/100 | Best Valid Score Until Now: 4.473 

Patience 2
Epoch: 9/10